Before starting, the "FinalDatasets" folder created by "DataPrepration" notebook should be copied to project directory. In case both Notebooks are in the same directory, nothing needs to be done.

## Setup and Imports

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict
import torch.optim as optim
import torch.nn.functional as F
from collections import Counter
import random
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchsummary import summary

In [2]:
STOCKS = ['BOM500875','BOM532939','BOM524715','BOM532215','BOM532648','BOM532500','BOM500470','BOM500570','BOM500112','BOM532540','BOM500209','BOM500295','BOM532174','BOM500180','BOM500696','BOM500510','BOM500247','BOM500520','BOM532921','BOM532977','BOM500182','BOM507685','BOM500010','BOM532454','BOM500820','BOM500312','BOM532898','BOM532187','BOM533278','BOM532555','BOM570001']

## Data Prepration and LSTM Model creation

In [3]:
# Helper function for creating input to the model given full dataframe
def create_dataset(stock_data_full, lookback=30):
  # Initialize empty lists for storing sequences and labels
  sequences = []
  labels = []

  # Loop over the data and create 30-day sequences with corresponding labels
  for i in range(len(stock_data_full) - lookback):
      # Get the 30-day sequence for this row, including all features
      sequence = stock_data_full.iloc[i:i+lookback].values
      # Append the sequence to the list
      sequences.append(np.float32(sequence))
      # Get the label for this sequence, which is the stock price on the next trading day
      label = stock_data_full.iloc[i+lookback]['Label']
      # Append the label to the list
      labels.append(label)

  # Convert the sequences and labels to numpy arrays
  X = np.array(sequences)
  y = np.array(labels)

  # Print the shapes of X and y to verify they are the correct dimensions
  print(X.shape)  # should be (num_sequences, window_size, num_features)
  print(y.shape)  # should be (num_sequences,)

  return torch.from_numpy(X), torch.tensor(y)

In [4]:
# Definition of LSTM model to be used for our traning 
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm1 = nn.LSTM(input_size, hidden_size[0], batch_first=True, dropout=0.2)
        self.lstm2 = nn.LSTM(hidden_size[0], hidden_size[1], batch_first=True, dropout=0.2)
        self.lstm3 = nn.LSTM(hidden_size[1], hidden_size[2], batch_first=True, dropout=0.2)
        self.linear = nn.Linear(hidden_size[2], output_size)
        
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size[0]).to(device)
        c0 = torch.zeros(1, x.size(0), self.hidden_size[0]).to(device)
        
        out, _ = self.lstm1(x, (h0, c0))
        out, _ = self.lstm2(out)
        out, _ = self.lstm3(out)
        out = self.linear(out[:, -1, :])
        
        return out

## Model Traning on 1 stock to check correctness

In [5]:
# Load one stocks dataset into a dataframe
df = pd.read_csv('./FinalDatasets/BOM500010Stock.csv')
df.set_index('Date', inplace=True)
df.sort_index(inplace=True)

In [6]:
# train-test split for time series
train_size = int(len(df) * 0.80)
test_size = len(df) - train_size
train, test = df[:train_size], df[train_size:]

In [7]:
# Convert DF to Tensors to be used as LSTM Input
lookback = 30
batch_size =32
X_train, y_train = create_dataset(train, lookback=lookback)
X_test, y_test = create_dataset(test, lookback=lookback)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
# Create training and testing datasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Create data loaders for batching the datasets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

(2566, 30, 26)
(2566,)
(619, 30, 26)
(619,)
torch.Size([2566, 30, 26]) torch.Size([2566])
torch.Size([619, 30, 26]) torch.Size([619])


In [8]:
from torch.types import Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the model hyperparameters
input_size = 26
hidden_size = [32, 16, 10]
output_size = 1
dropout_prob = 0.2

# Initialize the model
model = LSTMModel(input_size, hidden_size, output_size).to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [9]:
# setting up hyperparameters
learning_rate = 0.001

# Initialize the model and the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999))

# Define the loss function
criterion = nn.MSELoss()

In [10]:
num_epochs=20
# Train the LSTM model
for epoch in range(num_epochs):
    for i, (sequences, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(sequences.float()) # cast input tensor to Float
        loss = criterion(outputs, labels.float().unsqueeze(1)) # cast label tensor to Float
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

    # Print the loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [10/20], Loss: 0.6062
Epoch [20/20], Loss: 1.1472


In [11]:
correct = 0
total = 0
with torch.no_grad():
    for sequences, labels in test_loader:
        outputs = model(sequences)
        predicted = torch.round(outputs).flatten().int()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")


Accuracy on test set: 26.33%


## Traning pipeline for all 31 stocks

In [12]:
traningAccuracies = dict()

In [13]:
def trainAndTestModelOnStock(stock):
  fileName = stock + 'Stock.csv'
  df = pd.read_csv('./FinalDatasets/' + fileName)
  df.set_index('Date', inplace=True)
  df.sort_index(inplace=True)
  
  # train-test split for time series
  train_size = int(len(df) * 0.80)
  test_size = len(df) - train_size
  train, test = df[:train_size], df[train_size:]

  # Convert dataframe to 3-D tensors
  lookback = 30
  batch_size =32
  X_train, y_train = create_dataset(train, lookback=lookback)
  X_test, y_test = create_dataset(test, lookback=lookback)
  
  # Create training and testing datasets
  train_dataset = TensorDataset(X_train, y_train)
  test_dataset = TensorDataset(X_test, y_test)

  # Create data loaders for batching the datasets
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

  from torch.types import Device
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  # Define the model hyperparameters
  input_size = 26
  hidden_size = [32, 16, 10]
  output_size = 1
  dropout_prob = 0.2
  learning_rate = 0.001

  # Initialize the model
  model = LSTMModel(input_size, hidden_size, output_size).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999))
  criterion = nn.MSELoss()

  # Train the LSTM model
  num_epochs=20
  for epoch in range(num_epochs):
      for i, (sequences, labels) in enumerate(train_loader):
          optimizer.zero_grad()
          outputs = model(sequences.float()) # cast input tensor to Float
          loss = criterion(outputs, labels.float().unsqueeze(1)) # cast label tensor to Float
          loss.backward()
          nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
          optimizer.step()

      # Print the loss every 10 epochs
      if (epoch + 1) % 10 == 0:
          print(f"{stock} Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
  
  # Test the Model and save accuracy
  correct = 0
  total = 0
  with torch.no_grad():
      for sequences, labels in test_loader:
          outputs = model(sequences)
          predicted = torch.round(outputs).flatten().int()
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print(f"{stock} Accuracy on test set: {100 * correct / total:.2f}%")
  traningAccuracies[stock] = correct/total

In [14]:
for stock in STOCKS:
  trainAndTestModelOnStock(stock)

(2922, 30, 26)
(2922,)
(708, 30, 26)
(708,)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


BOM500875 Epoch [10/20], Loss: 0.7194
BOM500875 Epoch [20/20], Loss: 0.2344
BOM500875 Accuracy on test set: 59.18%
(2134, 30, 26)
(2134,)
(512, 30, 26)
(512,)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


BOM532939 Epoch [10/20], Loss: 0.3047
BOM532939 Epoch [20/20], Loss: 0.6245
BOM532939 Accuracy on test set: 49.02%
(2922, 30, 26)
(2922,)
(708, 30, 26)
(708,)


...

BOM533278 Epoch [10/20], Loss: 0.2533
BOM533278 Epoch [20/20], Loss: 0.2514
BOM533278 Accuracy on test set: 31.83%
(2789, 30, 26)
(2789,)
(675, 30, 26)
(675,)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


BOM532555 Epoch [10/20], Loss: 0.1303
BOM532555 Epoch [20/20], Loss: 0.2972
BOM532555 Accuracy on test set: 59.70%
(1910, 30, 26)
(1910,)
(456, 30, 26)
(456,)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


BOM570001 Epoch [10/20], Loss: 0.1596
BOM570001 Epoch [20/20], Loss: 0.3176
BOM570001 Accuracy on test set: 35.31%


In [15]:
print('Classification Accuracies for each of 31 stock on Testing set is as follows: ')

print ("{:<10} {:<5}".format('Stock','Accuracy'))
for k, v in traningAccuracies.items():
        accuracy = v
        print ("{:<8} {:<15}".format(k, accuracy))

Classification Accuracies for each of 31 stock on Testing set is as follows: 
Stock      Accuracy
BOM500875 0.5918079096045198
BOM532939 0.490234375    
BOM524715 0.5098870056497176
BOM532215 0.7926657263751763
BOM532648 0.5023400936037441
BOM532500 0.3624823695345557
BOM500470 0.4646892655367232
BOM500570 0.6370056497175142
BOM500112 0.4400564174894217
BOM532540 0.621897810218978
BOM500209 0.45557122708039494
BOM500295 0.4245416078984485
BOM532174 0.5260930888575458
BOM500180 0.23554301833568406
BOM500696 0.5133991537376587
BOM500510 0.5588652482269504
BOM500247 0.6050775740479548
BOM500520 0.5464788732394367
BOM532921 0.4099616858237548
BOM532977 0.6807228915662651
BOM500182 0.3921015514809591
BOM507685 0.3328631875881523
BOM500010 0.2859450726978998
BOM532454 0.5211864406779662
BOM500820 0.6248236953455572
BOM500312 0.5084507042253521
BOM532898 0.29867674858223064
BOM532187 0.2633802816901408
BOM533278 0.3183023872679045
BOM532555 0.597037037037037
BOM570001 0.3530701754385965


## Portfolio Optimization

In [ ]:
# TO BE DONE

In [ ]:
!pip freeze

absl-py==1.4.0
alabaster==0.7.13
albumentations==1.2.1
altair==4.2.2
anyio==3.6.2
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
array-record==0.2.0
arviz==0.15.1
astropy==5.2.2
astunparse==1.6.3
attrs==23.1.0
audioread==3.0.0
autograd==1.5
Babel==2.12.1
backcall==0.2.0
beautifulsoup4==4.11.2
bleach==6.0.0
blis==0.7.9
blosc2==2.0.0
bokeh==2.4.3
branca==0.6.0
build==0.10.0
CacheControl==0.12.11
cached-property==1.5.2
cachetools==5.3.0
catalogue==2.0.8
certifi==2022.12.7
cffi==1.15.1
chardet==4.0.0
charset-normalizer==2.0.12
chex==0.1.7
click==8.1.3
cloudpickle==2.2.1
cmake==3.25.2
cmdstanpy==1.1.0
colorcet==3.0.1
colorlover==0.3.0
community==1.0.0b1
confection==0.0.4
cons==0.4.5
contextlib2==0.6.0.post1
contourpy==1.0.7
convertdate==2.4.0
cryptography==40.0.2
cufflinks==0.17.3
cvxopt==1.3.0
cvxpy==1.3.1
cycler==0.11.0
cymem==2.0.7
Cython==0.29.34
dask==2022.12.1
datascience==0.17.6
db-dtypes==1.1.1
dbus-python==1.2.16
debugpy==1.6.6
decorator==4.4.2
defusedxml==0.7.1
di